In [ ]:
import sys
import os
sys.path.insert(0,'../../')
from scripts.main import *
from IPython.display import display, clear_output
import pandas as pd

In [ ]:
href_ = '/teams/ATL/2023/gamelog/'
page = PageHTML(href_,content_only=False).auto_instance()
print(page)
# page.load(from_ = '/data/')
# page.load(from_ = BaseDatabaseManager('/data/'))
page.fetch()
print(page)
# page.render()

In [ ]:
page = AllSeasonsIndexHTML()
page.fetch()
page.downstream_hrefs()

In [ ]:
page.downstream_hrefs()[27:]

In [ ]:
LOCAL_HOST = '/Volumes/Seagate Portable Disk/University of Manitoba/Data Science/Datasets/basketball-analytics/https:/www.basketball-reference.com'
db = DatabaseManager(LOCAL_HOST)
db.info()

In [ ]:
page = db.load('/leagues/NBA_1996.html',auto_instance=True)
page

In [ ]:
page.downstream_hrefs()

In [ ]:
# page = AllSeasonsIndexHTML()
# page.fetch(wait=2)
# # page.render()
# page.seasons_hrefs()

In [ ]:
# hrefs = [
#     '/', '/leagues/', '/teams/', '/players/',
#     '/boxscores/' , '/boxscores/?month=12&day=25&year=2022' ,
#     '/leagues/NBA_2023.html', '/leagues/NBA_2023_games.html', '/leagues/NBA_2023_games-november.html',
#     '/teams/GSW/', '/teams/GSW/2023.html', '/teams/GSW/2023/gamelog/', '/teams/GSW/2023/gamelog-advanced/',
#     '/players/a/', '/players/z/', 
#     '/players/c/curryst01.html' , '/players/j/jamesle01.html' ,
#     '/boxscores/202212250DAL.html', '/boxscores/202212250GSW.html' ,
#     '/boxscores/pbp/202212250DAL.html' , '/boxscores/pbp/202212250GSW.html' ,
#     '/boxscores/shot-chart/202212250DAL.html', '/boxscores/shot-chart/202212250GSW.html' ,
#     '/boxscores/plus-minus/202212250DAL.html', '/boxscores/plus-minus/202212250GSW.html'
# ]


# for href in hrefs:
#     page = PageHTML(href,content_only=False).auto_instance()
#     page.fetch()
#     print(page)
#     page.render()
#     time.sleep(5)
#     clear_output()
#     print(page)
#     page.content_only(True)
#     page.render()
#     time.sleep(5)
#     clear_output()


In [ ]:
class DatabaseManager(BaseDatabaseManager):
    # Contains functionality that syncs specific folders
    def sync_all_seasons_index(self,downstream=False,pull=True,new_files_only=False,warn=False,wait=3,position=0):   
        href = AllSeasonsIndexHTML().href_
        sync_results            = SyncResult()
        sync_status             = self.sync(href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait)
        sync_results[sync_status].append(href)
        if downstream:
            # sync season schedule
            all_seasons_index_page = self.load(href,auto_instance=True)
            season_index_hrefs = tqdm(all_seasons_index_page.downstream_hrefs(),leave=False,position=position+1)
            for season_index_href in season_index_hrefs:
                season_index_hrefs.set_description(season_index_href)
                sync_downstream_results = self.sync_season_index(season_index_href,downstream=downstream,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
                sync_results.update(sync_downstream_results)
            # sync other pages if necessary
        return sync_results

    
    def sync_season_index(self,href,downstream=False,pull=True,new_files_only=False,warn=False,wait=3,position=0):
        sync_results            = SyncResult()
        sync_status             = self.sync(href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
        sync_results[sync_status].append(href)

        if downstream:
            # sync season schedule
            season_index_page = self.load(href,auto_instance=True)
            season_schedule_hrefs = tqdm(season_index_page.downstream_hrefs(),leave=False,position=position+1)
            for season_schedule_href in season_schedule_hrefs:
                season_schedule_hrefs.set_description(season_schedule_href)
                sync_downstream_results = self.sync_season_schedule(season_schedule_href,downstream=downstream,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
                sync_results.update(sync_downstream_results)
        return sync_results
    
    def sync_season_schedule(self,href,downstream=False,pull=True,new_files_only=False,warn=False,wait=3,position=0):
        sync_results            = SyncResult()
        sync_status             = self.sync(href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
        sync_results[sync_status].append(href)
        
        if downstream:
            season_schedule_page = self.load(href,auto_instance=True)
            monthly_schedule_hrefs = tqdm(season_schedule_page.downstream_hrefs(),leave=False,position=position+1)
            for monthly_schedule_href in monthly_schedule_hrefs:
                monthly_schedule_hrefs.set_description(monthly_schedule_href)
                sync_downstream_results = self.sync_monthly_schedule(monthly_schedule_href,downstream=downstream,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
                sync_results.update(sync_downstream_results)
        return sync_results
    
    def sync_monthly_schedule(self,href,downstream=False,pull=True,new_files_only=False,warn=False,wait=3,position=0):
        sync_results            = SyncResult()
        sync_status             = self.sync(href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
        sync_results[sync_status].append(href)
        
        if downstream:
            monthly_schedule_page = self.load(href,auto_instance=True)
            match_boxscores_hrefs = tqdm(monthly_schedule_page.downstream_hrefs(),leave=False,position=position+1)
            for match_boxscores_href in match_boxscores_hrefs:
                match_boxscores_hrefs.set_description(match_boxscores_href)
                sync_downstream_results = self.sync_match_boxscores(match_boxscores_href,downstream=downstream,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
                sync_results.update(sync_downstream_results)
        return sync_results

    def sync_match_boxscores(self,href,downstream=False,pull=True,new_files_only=False,warn=False,wait=3,position=0):
        sync_results            = SyncResult()
        sync_status             = self.sync(href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
        sync_results[sync_status].append(href)

        if downstream:
            match_boxscores_page = self.load(href,auto_instance=True)
            match_sub_boxscores_hrefs = tqdm(match_boxscores_page.downstream_hrefs(),leave=False,position=position+1)
            for match_sub_boxscores_href in match_sub_boxscores_hrefs:
                match_sub_boxscores_hrefs.set_description(match_sub_boxscores_href)
                sync_status = self.sync(match_sub_boxscores_href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
                sync_results[sync_status].append(match_sub_boxscores_href)
        return sync_results


    def sync(self,href,pull=True,new_files_only=False,downstream=False,warn=False,wait=3,position=0):
        sync_results            = SyncResult()
        sync_status             = super().sync(href,pull=pull,new_files_only=new_files_only,warn=warn,wait=wait) # sync season_schedule_page 
        sync_results[sync_status].append(href)

        if downstream:
            this_level_page = self.load(href,auto_instance=True)
            downstream_hrefs = tqdm(this_level_page.downstream_hrefs(),leave=False,position=position+1)
            for downstream_href in downstream_hrefs:
                downstream_hrefs.set_description(downstream_href)
                sync_status = self.sync(downstream_href,pull=pull,new_files_only=new_files_only,downstream=downstream,warn=warn,wait=wait) # sync season_schedule_page 
                sync_results[sync_status].append(downstream_href)
        return sync_results

db_path = './demo2/'
bbref_db = DatabaseManager(db_path)
bbref_db.info()

In [ ]:
sync_result = bbref_db.sync('/leagues/NBA_2016_games-october.html',new_files_only=True,downstream=True,wait=4)
sync_result.dict()

In [ ]:
sync_result = bbref_db.sync_monthly_schedule('/leagues/NBA_2016_games-october.html',downstream=True,new_files_only=True,wait=2)
sync_result.dict()

In [ ]:
# page = PageHTML('/boxscores/202304290DEN.html').auto_instance()
# page.fetch()
# page.soup().select('div.filter div:not(.current) a:not(.sr_preset)')
# # page.render()

In [ ]:
page = bbref_db.load('/leagues/NBA_2016_games.html',auto_instance=True)
page.downstream_hrefs()

In [ ]:
from main import *
from IPython.display import display, clear_output

In [ ]:
LOCAL_HOST = '/Volumes/Seagate Portable Disk/University of Manitoba/Data Science/Datasets/basketball-analytics/https:/www.basketball-reference.com'
db = DatabaseManager(LOCAL_HOST)
print(db.info())

page = db.load('/leagues/NBA_2023.html')

In [ ]:
import ast

def parse(self,extract_hrefs=True):
    parsed_tables = {}
    for i,table in enumerate(self.soup().find('table')):
        table_id = 'table_'+i
        if table.get('id'):
            table_id = table.get('id')
        parsed_tables[table_id] = parse_table(table,extract_hrefs=extract_hrefs)
    return parsed_tables

def parse_table(table,extract_hrefs=True):
    if extract_hrefs:
        df = pd.concat(pd.read_html(str(table),flavor='bs4',extract_links='body'))
        if isinstance(df.columns,pd.MultiIndex):
            df = df.droplevel(0,axis=1)
        hrefs = []
        for col in df.columns:
            df_col  = df[col].apply(lambda x: ast.literal_eval(str(x))[0])
            df_href = df[col].apply(lambda x: ast.literal_eval(str(x))[1]).rename(col+'_href')
            hrefs.append(df_col)
            if df_href.notnull().any():
                hrefs.append(df_href)
        return pd.concat(hrefs,axis=1)
    else:
        df = pd.concat(pd.read_html(str(table),flavor='bs4',extract_links='body')).droplevel(0,axis=1)
        if isinstance(df.columns,pd.MultiIndex):
            df = df.droplevel(0,axis=1)
    return df
    



# page.render()

In [1]:
import sys
import os
sys.path.insert(0,'../../')
from scripts.main import *
from IPython.display import display, clear_output
import pandas as pd

LOCAL_HOST = '/Volumes/Seagate Portable Disk/University of Manitoba/Data Science/Datasets/basketball-analytics/https:/www.basketball-reference.com'
db = DatabaseManager(LOCAL_HOST)
print(db.sync('/leaders/',content_only=False))
page = db.load('/leaders/',content_only=False)
print(page)

SYNC_STT_UPDATED
PageHTML('https://www.basketball-reference.com/leaders', content_type=local)


In [5]:
iter_tables(page.parse(),sleep=3)

table_8: 100%|██████████| 9/9 [00:27<00:00,  3.02s/it]


In [7]:
page = db.load('/boxscores/197710180CLE.html')
page.render()

In [4]:
db.browse('/boxscores')

{'root': '/Volumes/Seagate Portable Disk/University of Manitoba/Data Science/Datasets/basketball-analytics/https:/www.basketball-reference.com/boxscores',
 'folders': ['/pbp', '/plus-minus', '/shot-chart'],
 'files': ['/.DS_Store',
  '/197710180CLE.html',
  '/197710180DET.html',
  '/197710180MIL.html',
  '/197710180NYK.html',
  '/197710180PHO.html',
  '/197710190BUF.html',
  '/197710190DEN.html',
  '/197710190GSW.html',
  '/197710190HOU.html',
  '/197710190IND.html',
  '/197710190PHI.html',
  '/197710190SAS.html',
  '/197710200NOJ.html',
  '/197710210CHI.html',
  '/197710210DEN.html',
  '/197710210GSW.html',
  '/197710210KCK.html',
  '/197710210NJN.html',
  '/197710210SEA.html',
  '/197710210WSB.html',
  '/197710220ATL.html',
  '/197710220BUF.html',
  '/197710220DET.html',
  '/197710220HOU.html',
  '/197710220IND.html',
  '/197710220MIL.html',
  '/197710220NYK.html',
  '/197710220PHI.html',
  '/197710220POR.html',
  '/197710230LAL.html',
  '/197710230SEA.html',
  '/197710240DET.html',


In [4]:
# page.render()

In [ ]:
table = page.soup().select('table#per_game-team')
df_hrefs = pd.concat(pd.read_html(str(table),flavor='bs4',extract_links='body'))
isinstance(df_hrefs.columns,pd.MultiIndex)

In [ ]:
ast.literal_eval("''")

In [ ]:
page.soup().find('table').__class__

In [ ]:
for table in page.soup().find_all('table'):
    print(table)
    time.sleep(5)
    clear_output()

In [ ]:
table = page.soup().find('table')
id_ = table.get('id')
# pbp_table_elmt = html_soup.find('table',{'id':'pbp'})
# player_href_table = pd.DataFrame({(a.text,a['href']) for a in pbp_table_elmt.find_all('a')},columns=['player','player_href'])
# return {'player-href': player_href_table}
df_hrefs = pd.concat(pd.read_html(str(table),flavor='bs4'))
df_hrefs

In [ ]:
import ast

def parse_table(table,extract_hrefs=True):
    if extract_hrefs:
        df = pd.concat(pd.read_html(str(table),flavor='bs4',extract_links='body')).droplevel(0,axis=1)
        hrefs = []
        for col in df.columns:
            df_col  = df[col].apply(lambda x: ast.literal_eval(str(x))[0])
            df_href = df[col].apply(lambda x: ast.literal_eval(str(x))[1]).rename(col+'_href')
            hrefs.append(df_col)
            if df_href.notnull().any():
                hrefs.append(df_href)
        return pd.concat(hrefs,axis=1)
    else:
        df = pd.concat(pd.read_html(str(table),flavor='bs4',extract_links='body')).droplevel(0,axis=1)

parse_table(table)